## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [1]:
# Imports

from agents import Agent, WebSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import os
from IPython.display import display, Markdown
from pprint import pprint
import requests
load_dotenv(override=True)

# Constants

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

## We will be making 4 Agents:

1. Search Agent - searches online given a search term using an OpenAI hosted tool
2. Planner Agent - given a query from the user, come up with searches
3. Report Agent - make a report on results
4. Push Agent - send a notification to the user's phone with a summary

## Our First Agent: Search Agent

Given a Search term, search for it on the internet and summarize results.

In [2]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [3]:
message = "What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of May 2025, several AI agent frameworks have gained prominence for their capabilities in developing sophisticated AI applications:

**LangChain**: A leading open-source framework for building applications powered by large language models (LLMs). It offers a modular architecture that simplifies complex workflows, including context management and multi-step tasks. LangChain integrates seamlessly with various LLMs, such as OpenAI and Hugging Face, and supports semantic search and API interactions. It's particularly suited for conversational AI, research tools, and document analysis. ([curotec.com](https://www.curotec.com/insights/top-ai-agent-frameworks/?utm_source=openai))

**LangGraph**: An extension of LangChain, LangGraph focuses on multi-agent systems capable of collaboration and adaptation. It provides tools for coordinating multiple agents, visual graph-based workflows, and advanced error-handling capabilities. LangGraph is ideal for applications like storytelling, multi-step chatbots, and strategic planning tools. ([curotec.com](https://www.curotec.com/insights/top-ai-agent-frameworks/?utm_source=openai))

**CrewAI**: Designed for orchestrating role-playing AI agents, CrewAI enables the creation of specialized agents that work together on complex tasks. It features a role-based agent architecture, dynamic task planning, and inter-agent communication protocols. CrewAI is well-suited for developing virtual product development teams and collaborative AI systems. ([chatbase.co](https://www.chatbase.co/blog/ai-agent-frameworks?utm_source=openai))

These frameworks are at the forefront of AI agent development, offering robust tools for creating intelligent, collaborative, and adaptable AI systems. 

Take a look at the trace

https://platform.openai.com/traces

## Our Second Agent: Planner Agent

Given a query, come up with 5 ideas for web searches that could be run.

Use Structured Outputs as our way to ensure the Agent provides what we need.

In [4]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 5

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# We use Pydantic objects to describe the Schema of the output

class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""

# We pass in the Pydantic object to ensure the output follows the schema

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4.1-mini",
    output_type=WebSearchPlan,
)

In [5]:

message = "What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

WebSearchPlan(searches=[WebSearchItem(reason='To find the latest popular AI agent frameworks as of May 2025', query='most popular AI agent frameworks May 2025'), WebSearchItem(reason='To identify which AI agent frameworks have been most successful recently', query='successful AI agent frameworks 2025'), WebSearchItem(reason='To obtain expert reviews and comparisons of AI agent frameworks', query='AI agent framework comparison 2025'), WebSearchItem(reason='To check recent developer and user community feedback on AI agent frameworks', query='AI agent frameworks user reviews 2025'), WebSearchItem(reason='To find technology articles and reports discussing AI agent trends in 2025', query='AI agent frameworks trends May 2025')])


## Our Third Agent: Writer Agent

Take the results of internet searches and make a report

In [6]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

## Our Fourth Agent: push notification

Just to show how easy it is to make a tool!

I'm using a nifty product called PushOver - to set this up yourself, visit https://pushover.net

In [7]:
@function_tool
def push(message: str):
    """Send a push notification with this brief message"""
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    return {"status": "success"}

In [8]:
push

FunctionTool(name='push', description='Send a push notification with this brief message', params_json_schema={'properties': {'message': {'title': 'Message', 'type': 'string'}}, 'required': ['message'], 'title': 'push_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000022F41856520>, strict_json_schema=True)

In [9]:
INSTRUCTIONS = """You are a member of a research team and will be provided with a short summary of a report.
When you receive the report summary, you send a push notification to the user using your tool, informing them that research is complete,
and including the report summary you receive"""


push_agent = Agent(
    name="Push agent",
    instructions=INSTRUCTIONS,
    tools=[push],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required")
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [10]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and send a push notification

In [11]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_push(report: ReportData):
    """ Use the push agent to send a notification to the user """
    print("Pushing...")
    result = await Runner.run(push_agent, report.short_summary)
    print("Push sent")
    return report

### Showtime!

In [12]:
query ="What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_push(report)  
    print("Hooray!")
display(Markdown(report.markdown_report))

Starting research...
Planning searches...
Will perform 5 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Pushing...
Push sent
Hooray!


# Popular and Successful AI Agent Frameworks as of May 2025

## Table of Contents
1. [Introduction](#introduction)
2. [Overview of AI Agent Frameworks](#overview-of-ai-agent-frameworks)
3. [Key AI Agent Frameworks](#key-ai-agent-frameworks)
    - [LangChain](#langchain)
    - [CrewAI](#crewai)
    - [Agno](#agno)
    - [LangGraph](#langgraph)
    - [Microsoft Semantic Kernel](#microsoft-semantic-kernel)
    - [Microsoft AutoGen](#microsoft-autogen)
    - [OpenAI Agents SDK](#openai-agents-sdk)
    - [LlamaIndex Agents](#llamaindex-agents)
4. [Challenges in AI Agent Frameworks](#challenges-in-ai-agent-frameworks)
5. [Trends Shaping AI Agent Frameworks](#trends-shaping-ai-agent-frameworks)
6. [Conclusion](#conclusion)
7. [References](#references)  

## 1. Introduction
The landscape of Artificial Intelligence (AI) is ever-evolving, with a surge in the popularity and utility of AI agent frameworks. As of May 2025, frameworks such as LangChain, CrewAI, and Microsoft's Semantic Kernel have emerged as leading players in the development of autonomous systems. This report aims to explore the characteristics, strengths, and application suitability of these frameworks, as well as the challenges and evolving trends in AI agent frameworks.  

## 2. Overview of AI Agent Frameworks  
AI agent frameworks offer developers the ability to create intelligent agents capable of performing a diverse array of tasks, from simple automation to complex, multi-agent collaborations. Each framework is designed with unique features, addressing specific needs ranging from enhanced flexibility and ease of use to robust performance and integration capabilities.  

## 3. Key AI Agent Frameworks  
In this section, we delve into some of the most notable AI agent frameworks that are shaping the field as of May 2025.  

### LangChain  
LangChain stands out as a versatile and modular framework designed for applications powered by large language models (LLMs). Its primary offerings include:  
- **Workflow Creation**: LangChain facilitates managing complex workflows, allowing for smooth transitions between tasks.  
- **Integration**: It can seamlessly connect with multiple LLMs and external APIs, enabling diverse applications such as conversational AI, automated research, and document analysis.  
- **Extensive Community**: The framework has built up a large community and extensive documentation, making it suitable for both beginners and experienced developers.

However, some users report that its high-level abstractions may introduce performance overhead, which can impact application speed in production environments.  

### CrewAI  
CrewAI focuses on orchestrating multi-agent collaborations, offering a role-based architecture that simplifies developing team-like systems:  
- **Role-Based Agent Architecture**: This allows for specialized agents to work on distinct tasks, emulating human teamwork.  
- **Task Coordination**: CrewAI features advanced task planning and inter-agent communication, making it suitable for collaborative AI systems.  
- **Accessibility**: Its user-friendly design minimizes the learning curve for new users.

Despite its best efforts, CrewAI has faced challenges regarding interpretability and scalability, which may limit its deployment in advanced applications.  

### Agno  
Formerly known as Phidata, Agno has emerged as a strong open-source competitor in the AI agent framework space:  
- **Multimodal Support**: Agno focuses on building multimodal agents that can function across various environments.  
- **Performance**: Developers appreciate its speed in agent instantiation and lower memory usage, ideal for resource-constrained applications.

### LangGraph  
Building on LangChain's capabilities, LangGraph introduces a graph-based architecture:
- **Stateful Multi-Agent Systems**: LangGraph enables the management of multi-step tasks with real-time task coordination, essential for complex workflows.  
- **Control Over Interactions**: Its explicit management of agent interactions allows for superior error handling and branching.  

However, the framework has a steeper learning curve, which may pose challenges for newcomers.  

### Microsoft Semantic Kernel  
Tailored for enterprise integration, the Semantic Kernel provides significant advantages:
- **Cross-Language Support**: The framework supports multiple programming languages, making it versatile for large-scale applications.  
- **Task Orchestration and Analytics**: Enterprises benefit from built-in analytics, ensuring compliance and security in their AI applications.

### Microsoft AutoGen  
AutoGen, another strong contender from Microsoft, facilitates the development of real-time, multi-agent systems:  
- **Real-Time Capabilities**: Suitable for high-performance applications such as financial trading and IoT monitoring.  

### OpenAI Agents SDK  
OpenAI's SDK packages core functionalities, enabling reasoned decisions, API calls, and multi-agent orchestrations:  
- **Streamlined Development**: It simplifies the agent development process, reducing complexity for developers.

### LlamaIndex Agents  
Integrating retrieval-augmented generation (RAG) with indexing enhances data synergy.  
- **Enhanced Retrieval**: This framework excels in scenarios involving extensive data retrieval and knowledge fusion, making it well-suited for research and analytics tasks.  

## 4. Challenges in AI Agent Frameworks  
Despite their advancements, various challenges persist within the AI agent framework landscape:  
- **Data Quality**: High-quality data is essential for training effective agents, and obtaining such data can often be difficult.  
- **Computing Constraints**: Performance optimizations may be required to operate within resource-limited settings.  
- **Ethical Concerns**: The implementation of AI raises ethical questions regarding fairness, privacy, and bias, necessitating robust governance frameworks.
- **Scalability**: Many frameworks face challenges in scaling to meet growing application demands, particularly in multi-agent contexts.

## 5. Trends Shaping AI Agent Frameworks  
Several trends are currently influencing the development and functionality of AI agent frameworks:
- **Autonomous Agents**: Increased capabilities for agents to perform complex tasks autonomously, emphasizing the rise of intelligent autonomy.  
- **Open Source Models**: The emergence of high-performance open-source models is disrupting traditional proprietary offerings, promoting inclusivity in development.  
- **Multimodal Interaction**: Frameworks integrating multiple media types (text, audio, video) create more versatile agents.  
- **Decentralization**: The shift toward decentralized models running locally enhances performance and privacy.  
- **Ethical AI**: An increase in frameworks focused on ensuring fairness and transparency reflects the growing importance of ethical considerations in AI development.
- **Unified AI Ecosystems**: Platforms that facilitate interaction among multiple agents are on the rise, streamlining processes and innovation.

## 6. Conclusion  
AI agent frameworks have gained considerable traction in 2025, with innovative solutions like LangChain, CrewAI, and Microsoft Semantic Kernel leading the way. Each framework has its strengths, weaknesses, and particular use cases, contributing to the broader evolution of AI technology. As challenges persist and trends emerge, developers and organizations must carefully analyze their choices to maximize the benefits of AI agents in their operations.  

## 7. References  
- [Vestra.ai](https://vestra.ai/blogs/top-8-ai-agent-frameworks-to-watch-in-2025?utm_source=openai)
- [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2024/07/ai-agent-frameworks/?utm_source=openai)
- [Medium.com](https://medium.com/%40mohitcharan04/comprehensive-comparison-of-ai-agent-frameworks-bec7d25df8a6?utm_source=openai)
- [Curotec.com](https://www.curotec.com/insights/top-ai-agent-frameworks/?utm_source=openai)
- [AADITECH](https://aaditech.com/2025/01/08/ai-agent-frameworks-for-2025/?utm_source=openai)
- [AragonResearch.com](https://aragonresearch.com/globe-for-ai-agent-platforms-2025/?utm_source=openai)
- [Wordware.ai](https://www.wordware.ai/blog/best-ai-agent-frameworks-for-developing-autonomous-systems?utm_source=openai)
- [Turing.com](https://www.turing.com/resources/ai-agent-frameworks?utm_source=openai)  

---  

This markdown report is structured to give a comprehensive insight into the state of AI agent frameworks in May 2025, detailing their functionalities, market presence, and the evolving landscape around them.

### As always, take a look at the trace

https://platform.openai.com/traces